# Post Trade Analysis

Run all cells to get an idea of the trade result.

In [ ]:
# Enable autoreload extension
%load_ext autoreload
%autoreload 2
import pandas as pd

from analysis.scripts.post_trade_plot import PostTradePlot

tool = PostTradePlot("/tmp/replay.sqlite")

In [ ]:
summary = pd.DataFrame.from_dict(tool.get_summary(), orient="index", columns=["Count"])
summary

In [ ]:
trade_result = tool.load_trade_result()
score_details = tool.wrangle(trade_result)
score_details["score"] = trade_result["opportunity.score"]
score_details["profit"] = trade_result["profit"]
score_details

In [ ]:
tool.draw().show()
tool.save_draw()

In [ ]:
tool.load_table("bull_trend_rider_opportunity")

# Machine Learning Experimental

In [ ]:
# model, predict_df = tool.predict(drop_columns=['min_prev_return_pct', 'max_prev_return_pct'])
# predict_tool = PostTradePlot("/tmp/replay.sqlite")
# predict_df = predict_tool.wrangle(predict_tool.load_trade_result(), drop_columns=['min_prev_return_pct', 'max_prev_return_pct'])
# predict_df["prediction"] = model.predict(predict_df.drop(columns=["is_profit"]))
# predict_df